In [1]:
# Build a fleshed out desktop exercise

# Target
# 47548 students
# 4137 teachers
# 3500 other
# 116 schools
import os
import sqlite3 as lite
from sqlalchemy import create_engine

from faker import Faker
from faker_e164.providers import E164Provider
import numpy as np
import pandas as pd
import random
import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

db_filename = r'mydb.db'

faker = Faker()
faker.add_provider(E164Provider)

DATE_END = datetime.datetime.now() + relativedelta(years=-25)
DATE_START = DATE_END + relativedelta(years=-34)

STU_DATE_END = datetime.datetime.now() + relativedelta(years=-6)
STU_DATE_START = STU_DATE_END + relativedelta(years=-12)

PC_DATE_END = datetime.datetime.now() + relativedelta(months=-2)
PC_DATE_START = PC_DATE_END + relativedelta(years=-3)
last_valid_date = datetime.datetime.now() + relativedelta(years=-30)
last_valid_student_date = datetime.datetime.now() + relativedelta(years=-6)
last_valid_purchase_date = datetime.datetime.now() + relativedelta(months=-3)

hs_cut_off = datetime.datetime.today() + relativedelta(years=-14)
ms_cut_off = datetime.datetime.today() + relativedelta(years=-11)

o1 = 10
o2 = random.randrange(0,200)


In [2]:
# Routines
# create_school
# create_employee
# create_devices

def create_school(schooltype):
    name = faker.name() + " " + schooltype
    address = faker.building_number() + " " + faker.street_name() + " " + faker.street_suffix()
    phone = faker.safe_e164(region_code="US")
    fax= faker.safe_e164(region_code="US")
    while phone == fax:
        fax= faker.safe_e164(region_code="US")
    return name, address, phone, fax

def create_employee(emptype):
    global last_valid_date
    if emptype == "principal":
        education = "doctorate"
        edschool = "University"
    elif emptype == "teacher":
        education = random.choice(["bachelors", "masters", "bachelors", "masters", "bachelors", "bachelors", "doctorate"])
        if education != "bachelors":
            edschool = "University"
        else:
            edschool = random.choice(["College", "University"])
    else:
        education = random.choice(["associates", "bachelors"])
        if education != "bachelors":
            edschool = "Community College"
        else:
            edschool = random.choice(["College", "University"])
    edschool = faker.name() + " " + edschool
    fname = faker.first_name()
    lname = faker.last_name()
    address = faker.building_number() + " " + faker.street_name() + " " + faker.street_suffix()
    wphone = faker.safe_e164(region_code="US")
    hphone = faker.safe_e164(region_code="US")
    mphone = faker.safe_e164(region_code="US")
    while wphone == hphone:
        hphone= faker.safe_e164(region_code="US")
    while wphone == mphone:
        mphone= faker.safe_e164(region_code="US")
    try:
        dob = faker.date_between_dates(DATE_START, DATE_END)
    except:
        dob = last_valid_date
    
    last_valid_date = dob
    ssn = faker.ssn()
    assignedschool = ""
    return fname, lname, address, wphone, hphone, mphone, dob, ssn, education, edschool, assignedschool

def create_student():
    global last_valid_student_date

    fname = faker.first_name()
    lname = faker.last_name()
    address = faker.building_number() + " " + faker.street_name() + " " + faker.street_suffix()
    hphone = faker.safe_e164(region_code="US")
    try:
        dob = faker.date_between_dates
        dob = faker.date_between_dates(STU_DATE_START, STU_DATE_END)
    except:
        dob = last_valid_student_date
    
    last_valid_date = dob
    ssn = faker.ssn()
    
    print(relativedelta(hs_cut_off,dob).years)
    
    if dob > hs_cut_off.date():
        school = random.randrange(0,16)
    elif dob > ms_cut_off.date():
        school = random.randrange(16,48)
    else:
        school = random.randrange(48,116)
    
    grade = relativedelta(datetime.date.today(), dob).years - 5 
    
    assignedschool = school
    return fname, lname, address, hphone, dob, ssn, grade, assignedschool

def create_device(dev_type):
    global last_valid_purchase_date
    asset_tag = faker.hexify(text='^^^^^^')
    if dev_type == "ipad":
        # ipad stuff
        make = "Apple"
        model = random.choice(["iPad Mini", "iPad"])
    elif dev_type == "desktop":
        # desktop pc stuff
        make = "Dell"
        model = random.choice(["Inspiron","Optiplex"])
    else:
        # laptop
        make = "Dell"
        model = random.choice(["Latitude 3500","Latitude 3310","Latitude 3400"])        
    try:
        purchase_date = faker.date_between_dates(PC_DATE_START, PC_DATE_END)
    except:
        purchase_date = last_valid_purchase_date
    last_valid_purchase_date = purchase_date
    serial_number = faker.uuid4()
    mac_addr = faker.hexify(text='^^:^^:^^:^^:^^:^^')
    status = "unassigned"
    usage = "noone"
    return asset_tag, make, model, purchase_date, serial_number, mac_addr, status, usage

def create_student():
    global last_valid_student_date

    fname = faker.first_name()
    lname = faker.last_name()
    address = faker.building_number() + " " + faker.street_name() + " " + faker.street_suffix()
    hphone = faker.safe_e164(region_code="US")
    try:
        dob = faker.date_between_dates
        dob = faker.date_between_dates(STU_DATE_START, STU_DATE_END)
    except:
        dob = last_valid_student_date
    
    last_valid_date = dob
    ssn = faker.ssn()
    
    if dob > hs_cut_off.date():
        school = random.randrange(0,16)
    elif dob > ms_cut_off.date():
        school = random.randrange(16,48)
    else:
        school = random.randrange(48,116)
    
    grade = relativedelta(datetime.date.today(), dob).years - 5 
    
    assignedschool = school
    return fname, lname, address, hphone, dob, ssn, grade, assignedschool

def find_avail_device(devtype, df_devices):
    device = df_devices[(df_devices['dev_type'] == devtype) & (df_devices['usage']=='noone')].head(1) 
    i = device.index.astype(str).astype(int)
    return i[0]

def find_school_wo_leader():
    mask = (df_school['principal'] == '') 
    return mask.idxmax()

In [3]:
# create the schools and put in a dataset
city = faker.city()
state = "MD"

domain = city.replace(" ", "") + ".edu"

common_sites = ["www.google.com", "www.facebook.com", "www.twitter.com", "www.espn.com", "www.instagram.com", "canvas."+domain, "www.pandora.com", "www.spotify.com", "www.netflix.com", "www.ticketmaster.com"]
business_sites = ["www.mysis.com", "admin.mysis.com", "www.linkedin.com", "www.staples.com", "www.amazon.com", "www.airbnb.com", "www.expedia.com", "www.citibank.com", "www.bankofamerica.com", "admin."+domain, "portal."+domain]

df_school = pd.DataFrame(columns={"schoolid", "name", "address", "city", "state", "phone", "fax", "principal"})

for i in tqdm(range (0,16)):
    name, address, phone, fax = create_school("High School")
    values_to_add = {"schoolid": i+1, "name": name, "address": address, "city": city, "state": state, "phone": phone, "fax": fax, "principal":""}
    row_to_add = pd.Series(values_to_add, name=str(i+1))
    df_school = df_school.append(row_to_add)
for i in tqdm(range (16,48)):
    name, address, phone, fax = create_school("Middle School")
    values_to_add = {"schoolid": i+1, "name": name, "address": address, "city": city, "state": state, "phone": phone, "fax": fax, "principal":""}
    row_to_add = pd.Series(values_to_add, name=str(i+1))
    df_school = df_school.append(row_to_add)
for i in tqdm(range (48,116)):
    name, address, phone, fax = create_school("Elementary School")
    values_to_add = {"schoolid": i+1, "name": name, "address": address, "city": city, "state": state, "phone": phone, "fax": fax, "principal":""}
    row_to_add = pd.Series(values_to_add, name=str(i+1))
    df_school = df_school.append(row_to_add)
    
# save the dataset
df_school = df_school[["schoolid","name","address","city","state","phone","fax","principal"]]

con = lite.connect(db_filename)
df_school.to_sql('schools', con, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)
con.close()

100%|█████████████████████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 262.49it/s]


In [4]:
# Create the devices and put in a dataset (increase by order of magnitude for production)
df_devices = pd.DataFrame(columns={"asset_tag", "make", "model", "dev_type", "purchase_date", "serial_number", "mac_addr","status","usage"})
for i in tqdm(range(0,50000)):
    asset_tag, make, model, purchase_date, serial_number, mac_addr, status, usage = create_device("ipad")
    values_to_add = {"asset_tag": asset_tag, "make":make, "model":model, "dev_type":"tablet", "purchase_date":purchase_date, "serial_number":serial_number, "mac_addr":mac_addr,"status":status,"usage":usage}
    row_to_add = pd.Series(values_to_add, name=str(i))
    df_devices = df_devices.append(row_to_add)
    
for i in tqdm(range(50000, 55000)):
    asset_tag, make, model, purchase_date, serial_number, mac_addr, status, usage = create_device("laptop")
    values_to_add = {"asset_tag": asset_tag, "make":make, "model":model, "dev_type":"laptop", "purchase_date":purchase_date, "serial_number":serial_number, "mac_addr":mac_addr,"status":status,"usage":usage}
    row_to_add = pd.Series(values_to_add, name=str(i))
    df_devices = df_devices.append(row_to_add)    
    
for i in tqdm(range(55000, 58500)):
    asset_tag, make, model, purchase_date, serial_number, mac_addr, status, usage = create_device("desktop")
    values_to_add = {"asset_tag": asset_tag, "make":make, "model":model, "dev_type":"desktop", "purchase_date":purchase_date, "serial_number":serial_number, "mac_addr":mac_addr,"status":status,"usage":usage}
    row_to_add = pd.Series(values_to_add, name=str(i))
    df_devices = df_devices.append(row_to_add)
    


100%|██████████████████████████████████████████████████████████████████████████████| 3500/3500 [00:57<00:00, 61.11it/s]


In [5]:
# Create the employees, assign them a device and assign them to schools
df_emp = pd.DataFrame(columns={"empid", "fname", "lname", "address", "city", "state", "wphone", "hphone", "mphone", "dob", "ssn", "degreeattained", "degreeschool","schoolassignment", "deviceassignment"})
i = 1023
fname, lname, address, wphone, hphone, mphone, dob, ssn, education, edschool, assignedschool = create_employee("superintendant")
device = find_avail_device('laptop', df_devices)
a_tag = df_devices.iloc[device]['asset_tag']
values_to_add = {
    "empid": i, 
    "fname": fname, 
    "lname": lname, 
    "address": address, 
    "city": city, 
    "state": state, 
    "wphone": wphone, 
    "hphone": hphone, 
    "mphone": mphone, 
    "dob": dob, 
    "ssn": ssn, 
    "degreeattained": education,
    "degreeschool": edschool,
    "schoolassignment": "superintendant",
    "deviceassignment": a_tag
}
df_devices.status[df_devices['asset_tag']==a_tag] = 'assigned'
df_devices.usage[df_devices['asset_tag']==a_tag] = i
row_to_add = pd.Series(values_to_add, name=1)
df_emp = df_emp.append(row_to_add)

for i in tqdm(range(1, 116)):
    fname, lname, address, wphone, hphone, mphone, dob, ssn, education, edschool, assignedschool = create_employee("teacher")
    device = find_avail_device('laptop', df_devices)
    a_tag = df_devices.iloc[device]['asset_tag']
    school = find_school_wo_leader()
    values_to_add = {
        "empid": i + 1023, 
        "fname": fname, 
        "lname": lname, 
        "address": address, 
        "city": city, 
        "state": state, 
        "wphone": wphone, 
        "hphone": hphone, 
        "mphone": mphone, 
        "dob": dob, 
        "ssn": ssn, 
        "degreeattained": education,
        "degreeschool": edschool,
        "schoolassignment": school,
        "deviceassignment": a_tag
    }
    df_devices.status[df_devices['asset_tag']==a_tag] = 'assigned'
    df_devices.usage[df_devices['asset_tag']==a_tag] = i + 1023
    df_school.principal[df_school['schoolid']==school] = i + 1023
    row_to_add = pd.Series(values_to_add, name=1)
    df_emp = df_emp.append(row_to_add)
    
k= 116

for i in tqdm(range(1140,1140+464)):
    school = (i-1140)//4 + 1
    if school > 116:
        school = random.randrange(1,116)
    fname, lname, address, wphone, hphone, mphone, dob, ssn, education, edschool, assignedschool = create_employee("staff")
    device = find_avail_device('desktop', df_devices)
    a_tag = df_devices.iloc[device]['asset_tag']
    values_to_add = {
        "empid": i, 
        "fname": fname, 
        "lname": lname, 
        "address": address, 
        "city": city, 
        "state": state, 
        "wphone": wphone, 
        "hphone": hphone, 
        "mphone": mphone, 
        "dob": dob, 
        "ssn": ssn, 
        "degreeattained": education,
        "degreeschool": edschool,
        "schoolassignment": school,
        "deviceassignment": device
    }
    df_devices.status[df_devices['asset_tag']==a_tag] = 'assigned'
    df_devices.usage[df_devices['asset_tag']==a_tag] = i 
    row_to_add = pd.Series(values_to_add, name=i)
    df_emp = df_emp.append(row_to_add)        

df_emp = df_emp[["empid", "fname", "lname", "address", "city", "state", "wphone", "hphone", "mphone", "dob", "ssn", "degreeattained", "degreeschool","schoolassignment", "deviceassignment"]]

con = lite.connect(db_filename)
df_school.to_sql('employees', con, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)
con.close()

100%|████████████████████████████████████████████████████████████████████████████████| 464/464 [00:19<00:00, 23.43it/s]


In [6]:
# Create the student, assign them a device and assign them to schools
df_student = pd.DataFrame(columns={"stuid", "fname", "lname", "address", "city", "state", "hphone", "dob", "ssn", "grade","schoolassignment", "deviceassignment"})
i = 311023
for i in tqdm(range(i, i+4780)):
    fname, lname, address, hphone, dob, ssn, grade, assignedschool = create_student()
    device = find_avail_device('tablet', df_devices)
    a_tag = df_devices.iloc[device]['asset_tag']
    values_to_add = {
        "stuid": i, 
        "fname": fname, 
        "lname": lname, 
        "address": address, 
        "city": city, 
        "state": state, 
        "hphone": hphone, 
        "dob": dob, 
        "ssn": ssn, 
        "grade": grade,
        "degreeschool": edschool,
        "schoolassignment": assignedschool,
        "deviceassignment": a_tag
    }
    df_devices.status[df_devices['asset_tag']==a_tag] = 'assigned'
    df_devices.usage[df_devices['asset_tag']==a_tag] = i
    row_to_add = pd.Series(values_to_add, name=i)
    df_student = df_student.append(row_to_add)

df_devices = df_devices[["asset_tag", "make", "model", "dev_type", "purchase_date", "serial_number", "mac_addr","status","usage"]]
df_student = df_student[["stuid", "fname", "lname", "address", "city", "state", "hphone", "dob", "ssn", "grade","schoolassignment", "deviceassignment"]]

con = lite.connect(db_filename)
df_devices.to_sql('devices', con, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)
df_student.to_sql('students', con, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)
con.close()

100%|██████████████████████████████████████████████████████████████████████████████| 4780/4780 [04:26<00:00, 17.91it/s]


In [7]:
def getHex(num):
    str = hex(num)
    pos = str.index('x') + 1
    str = str[pos:].zfill( 4)
    first = int(str[:2], 16)
    second = int(str[2:], 16)
    return first, second

In [8]:
o3, o4 = getHex(1024)
print(o1, o2, o3, o4)

10 170 4 0


In [ ]:
x = random.randint(1,101)
index = 301
if x < 80:
    machine 
    

In [ ]:
engine = create_engine(r"sqlite:///{}".format(db_filename))
sql = 'SELECT * from devices'
device_df = pd.read_sql(sql, engine)
for i in range(len(device_df)):
    print(device_df.iloc[i])

In [ ]:
# dhcp log
# ID Date,Time,Description,IP Address,Host Name,MAC Address
# 00,04/19/99,12:43:06,Started,,,
# 60,04/19/99,12:43:21,No DC is DS Enabled,,MYDOMAIN,
# 63,04/19/99,12:43:28,Restarting rogue detection,,,
# 01,04/19/99,13:11:13,Stopped,,,
# 00,04/19/99,12:43:06,Started,,,
# 55,04/19/99,12:43:54,Authorized(servicing),,MYDOMAIN,
        
# web logs
# from weblog.txt in cyber_data_maker
# 2019-06-01 09:00:00 GET 192.168.30.231 frys.com lpitwvtjjb.png 209.31.22.39 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
# 2019-06-01 09:02:43 GET 192.168.30.237 novamov.com eweznxpjgg.js 91.220.176.248 Mozilla/5.0 (iPhone; CPU iPhone OS 12_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Mobile/15E148 Safari/604.1
        
# application logs
# timestamp       user       action     ipaddress
# 2020-05-01      zrichards  login      4.4.2.3

# network logs
# $ last
# shs  pts/1  192.168.0.15  Mon Mar 19 17:48   still logged in
# shs  tty2   /dev/tty2     Mon Mar 19 17:37   still logged in
# shs  pts/2  192.168.0.15  Mon Mar 19 17:22 - 17:23  (00:00)
# jdoe pts/3  192.168.0.15  Mon Mar 19 16:51 - 17:22  (00:31)

#         Username    Port     From             Latest
# root        pts/0    boson.parts.org  Tue Jul 22 21:56:07 -0400 2014
# bin                                   **Never logged in**
# daemon                                **Never logged in**
# adm                                   **Never logged in**
# lp                                    **Never logged in**
# …
# shs         pts/6    204.123.45.67    Sun Apr 26 12:38:53 -0400 2015
# mia         pts/1    10.11.12.123     Mon Dec 17 11:15:07 -0500 2012

# wifi logs / wifi gateway logs

# LAN IP Address       Destination URL       Service or Port Number
# 192.168.5.108        58.11.186.216         6063
